In [51]:
import pandas as pd
from math import isnan
import duckdb
pd.set_option("display.precision", 3)
pd.set_option("display.max_columns", 30)

In [1]:
from utils.betting import (
	pivot_lines,
	value_count_plus,
	get_results,
	get_pp_close,
)

In [2]:
from sim_utils.modeling import update_config

In [53]:
pp = get_pp_close()
pp = pp.loc[pp.league_name == 'NFL'].copy(deep=True).reset_index(drop=True)

In [54]:
con = duckdb.connect("C:/Github/nfl_sim/data/nfl.duckdb")

In [55]:
con.execute("""select * from weekly
             where pat_made > 0 and season = 2025 limit 5""").fetchdf()

,game_id,gameday,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,team,opponent_team,completions,attempts,...,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance,fantasy_points,fantasy_points_ppr
0,2025_01_HOU_LA,2025-09-07,00-0039750,J.Karty,Joshua Karty,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,1,REG,LA,HOU,0,0,...,0,0,0,2,2,0,0,1.0,0,0,0,0,0,0.0,0.0
1,2025_03_MIA_BUF,2025-09-18,00-0036816,R.Patterson,Riley Patterson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2025,3,REG,MIA,BUF,0,0,...,0,0,0,3,3,0,0,1.0,0,0,0,0,0,0.0,0.0
2,2025_01_MIA_IND,2025-09-07,00-0039576,S.Shrader,Spencer Shrader,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,1,REG,IND,MIA,0,0,...,135,0,0,3,3,0,0,1.0,0,0,0,0,0,0.0,0.0
3,2025_03_MIA_BUF,2025-09-18,00-0023853,M.Prater,Matt Prater,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,3,REG,BUF,MIA,0,0,...,48,39,0,4,4,0,0,1.0,0,0,0,0,0,0.0,0.0
4,2025_01_CAR_JAX,2025-09-07,00-0039409,C.Little,Cam Little,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,1,REG,JAX,CAR,0,0,...,146,0,0,2,2,0,0,1.0,0,0,0,0,0,0.0,0.0


In [56]:
with open("./data/queries/pp_new_stats.sql", "r") as f:
    sql = f.read()

games = con.execute(sql).fetchdf()

In [57]:
games.loc[games.game_id == '2024_18_WAS_DAL']

,game_id,gsis_id,pbp_name,name,yards_first_catch,yards_2_catches,longest_reception,first_rush,yards_5_rushes,longest_rush,yards_first_rush,qtrs_over_5_rush_yards,qtrs_over_5_rec_yards,qtrs_with_reception,game_id_1,...,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
9234,2024_18_WAS_DAL,00-0039910,J.Daniels,Jayden Daniels,NaN,NaN,NaN,7.0,NaN,16.0,7.0,2.0,0.0,0.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
16191,2024_18_WAS_DAL,00-0039410,R.Flournoy,Ryan Flournoy,4.0,14.0,10.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
20753,2024_18_WAS_DAL,00-0039062,J.Mingo,Jonathan Mingo,14.0,24.0,14.0,NaN,NaN,NaN,NaN,0.0,1.0,2.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
22826,2024_18_WAS_DAL,00-0038738,H.Luepke,Hunter Luepke,NaN,NaN,NaN,10.0,NaN,10.0,10.0,1.0,0.0,0.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
22990,2024_18_WAS_DAL,00-0038622,D.Vaughn,Deuce Vaughn,NaN,NaN,NaN,3.0,36.0,12.0,3.0,2.0,0.0,0.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
25109,2024_18_WAS_DAL,00-0038547,L.Schoonmaker,Luke Schoonmaker,6.0,16.0,10.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
26994,2024_18_WAS_DAL,00-0038041,J.Ferguson,Jake Ferguson,19.0,12.0,19.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
27173,2024_18_WAS_DAL,00-0037801,K.Turpin,KaVontae Turpin,33.0,40.0,33.0,5.0,NaN,5.0,5.0,0.0,2.0,2.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
28829,2024_18_WAS_DAL,00-0037746,B.Robinson,Brian Robinson,6.0,NaN,6.0,3.0,10.0,6.0,3.0,1.0,0.0,1.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium
28876,2024_18_WAS_DAL,00-0037666,J.Tolbert,Jalen Tolbert,31.0,38.0,32.0,NaN,NaN,NaN,NaN,0.0,3.0,3.0,2024_18_WAS_DAL,...,-102,1,closed,matrixturf,NaN,NaN,00-0039910,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium


In [58]:
con.close()

In [ ]:
pp2nfl = {
    "FG": "fg_made",
    "INT": "passing_interceptions",
    "PAT Made": "pat_made",
    "Pass Attempts": "attempts",
    "Pass Completions": "completions",
    "Pass Deflections": "def_pass_defended",
    "Pass TDs": "passing_tds",
    "Pass Yards": "passing_yards",
    "Rec Targets": "targets",
    "Receiving Yards": "receiving_yards",
    "Receptions": "receptions",
    "Rush TDs": "rushing_tds",
    "Rush Yards": "rushing_yards",
    "Sacks": "def_sacks",
    "Sacks Taken": "sacks_suffered",
    "Receiving Yards in First 2 Receptions": "yards_2_catches",
    "Rush Yards in First 5 Attempts": "yards_5_rushes",
    "Quarters With 5+ Rush Yards": "qtrs_over_5_rush_yards",
    "Quarters With 1+ Reception": "qtrs_with_reception",
    "Longest Rush": "longest_rush",
    "Longest Reception": "longest_reception",
    "Rush Attempts": "carries",
    "Pass+Rush Yds": "passing_yards+rushing_yards",
    "Pass+Rush+Rec TDs": "passing_tds+rushing_tds+receiving_tds",
    "AST": "def_tackle_assists",
    "Yards on First Rush Attempt": "yards_first_rush",
    "Tackles For Loss": "def_tackles_for_loss",
    "Yards on First Reception": "yards_first_catch",
    "Quarters with 5+ Rush Yards": "qtrs_over_5_rush_yards",
    "Quarters with 10+ Receiving Yards": "qtrs_over_10_rec_yards",
}

In [60]:
pp['stat_nfl'] = pp['stat'].apply(lambda x: pp2nfl.get(x,x))
games = games.rename({'player_display_name':'player', 'opponent_team':'opp', 'gameday':'date'},axis=1)
games['date'] = pd.to_datetime(games['date'])

In [61]:
games['game_line'] = games.apply(lambda x: x['spread_line'] if x['home_team'] == x['team'] else -x['spread_line'],axis=1)
games['Rush+Rec TDs'] = games['receiving_tds'] + games['rushing_tds']
games['Completion Percentage'] = 100 * (games['completions'] / games['attempts'])
games['Pass+Rush Yds'] = games['passing_yards'] + games['rushing_yards']
games['Rush+Rec Yds'] = games['receiving_yards'] + games['rushing_yards']
games['Rush Yards Per Carry'] = games['rushing_yards'] / games['carries']
games['Kicking Points'] = games['pat_made'] + (3 * games['fg_made'])
games['Fantasy Score'] = ((games['passing_yards'] * 0.04) + (games['passing_tds'] * 4) + (2 * -games['passing_interceptions']) + (games['rushing_yards'] * 0.1) + (games['receiving_yards'] * 0.1) + #missing 2 pt conv
                    (games['receptions']) + (2 * (-games['receiving_fumbles_lost'] - games['rushing_fumbles_lost'] )) + (games['Rush+Rec TDs'] * 6) + 
                    games['pat_made'] + (3 * games['fg_made']) + games['fg_made_40_49'] + (games['fg_made_50_59'] * 2) + (games['fg_made_60_'] * 3) - games['pat_missed']  - games['fg_missed'] )

In [62]:
ref2pp = {
    'GNB':'GB',
    'JAX':'JAC',
    'KAN':'KC',
    'LVR':'LV',
    'NOR':'NO',
    'NWE':'NE',
    'SFO':'SF',
    'TAM':'TB',
}
games['team'] = games['team'].apply(lambda x: ref2pp.get(x,x))


def get_results(lines, games):
	def calc_outcome(row):
		outcome = 0
		for part in row['stat_nfl'].split('+'):
			try:
				outcome += row[part]
			except KeyError:
				return float('nan')
		return outcome

	comb = lines.merge(games, on=['player', 'date', 'team', 'opp'])
	comb['result'] = comb.apply(lambda row: calc_outcome(row), axis=1)
	comb['o_u'] = [
		'under' if x > y else 'over' if x < y else 'push' if x == y else 'N/A'
		for x, y in zip(comb['line'], comb['result'])
	]
	return comb.loc[comb.o_u != 'N/A'].reset_index(drop=True)

df = get_results(pp, games)
df.sample(3)

,player,team,line,alt_line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,__index_level_0__,...,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,game_line,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Rush Yards Per Carry,Kicking Points,Fantasy Score,o_u
19250,Devin Singletary,NYG,5.50,standard,Receiving Yards,ATL,9,NFL,2024-12-22 18:00:00+00,210959,2024-12-22,2024-12-22 17:40:00+00,484b8ce4-1,d08b6789-2,2834.0,...,Michael Penix Jr.,Brian Daboll,Raheem Morris,Tra Blake,ATL97,Mercedes-Benz Stadium,-9.5,0,NaN,12,12,3.000,0,1.2,under
40808,Dayo Odeyingbo,CHI,0.25,demon,Sacks,MIN,9,NFL,2025-09-09 00:00:00+00,260111,2025-09-08,2025-09-09 00:10:00+00,8fb200ca-9,77cd9760-3,505.0,...,Caleb Williams,Kevin O'Connell,Ben Johnson,Alan Eck,CHI98,Soldier Field,-1.5,0,NaN,0,0,NaN,0,0.0,over
629,J.K. Dobbins,LAC,14.50,standard,Receiving Yards,ARI,9,NFL,2024-10-22 01:00:00+00,211419,2024-10-21,2024-10-22 00:40:00+00,0863dc62-9,513ca492-0,723.0,...,Kyler Murray,Jim Harbaugh,Jonathan Gannon,Clete Blakeman,PHO00,State Farm Stadium,1.0,0,NaN,40,66,2.857,0,9.6,over


In [73]:
from math import floor


def get_tiers(value):
    if isnan(value):
        return value
    else:
        return floor(value / 10) * 10


def tiers(value):
    if isnan(value):
        return value
    if value <= -4.5:
        return "big L"
    if value <= 4.5:
        return "even"
    else:
        return "big W"


df["favored"] = df["game_line"].apply(lambda x: tiers(x))
# df['bucket'] = df['yard_pct'].apply(lambda x: get_tiers(x))
subset = df.loc[(df.alt_line != "goblin") & (df["date"] > "2024-10-01")].copy()
result = pivot_lines(subset, ["alt_line", "stat"]).sort_values(
    by="over_ratio", ascending=False
)
result.loc[(result["count"] > 5) & (result["alt_line"] == "standard")].sort_values(
    by=["over_ratio"], ascending=False
).drop(["over", "push", "under"], axis=1).head(20)

o_u,alt_line,stat,over_ratio,count
36,standard,Pass+Rush+Rec TDs,0.667,23.0
33,standard,Pass TDs,0.547,107.0
27,standard,Longest Reception,0.542,1813.0
26,standard,Kicking Points,0.540,427.0
22,standard,Completion Percentage,0.527,102.0
35,standard,Pass+Rush Yds,0.516,436.0
31,standard,Pass Completions,0.515,458.0
34,standard,Pass Yards,0.505,473.0
45,standard,Rush Yards in First 5 Attempts,0.502,237.0
23,standard,FG,0.500,192.0


In [71]:
df['stat'].value_counts()

stat
Receiving Yards                          12086
Receptions                                5918
Rush Yards                                5831
Fantasy Score                             2890
Pass Yards                                2794
Longest Reception                         2751
Rec Targets                               1811
Rush Attempts                             1554
Sacks                                     1208
Pass TDs                                  1090
Pass+Rush Yds                              925
FG                                         912
Longest Rush                               826
Pass Attempts                              786
INT                                        785
Pass Completions                           761
Kicking Points                             545
Rush Yards in First 5 Attempts             283
Receiving Yards in First 2 Receptions      210
Completion Percentage                      134
PAT Made                                    76
Sacks Ta

In [64]:
res = pivot_lines(df.loc[df.date > '2025-01-01'], ['stat_nfl', 'alt_line', 'favored']).sort_values(by='over_ratio').copy()
res.loc[res.stat_nfl=='Kicking Points']

o_u,stat_nfl,alt_line,favored,over,push,under,over_ratio,count
6,Kicking Points,demon,even,1.0,0.0,1.0,0.292,2.0
8,Kicking Points,standard,big L,11.0,3.0,10.0,0.516,24.0
9,Kicking Points,standard,big W,13.0,2.0,9.0,0.562,24.0
7,Kicking Points,goblin,even,1.0,0.0,1.0,0.625,2.0
10,Kicking Points,standard,even,35.0,3.0,18.0,0.635,56.0


In [65]:
working = list(res['stat_nfl'].unique())
available = list(pp['stat_nfl'].unique())
[x for x in available if x not in working and 'Combo' not in x]

['Rush+Rec TDs',
 'Rush+Rec Yds',
 'Tackles+Ast',
 'Punts Inside 20',
 'Completions in First 10 Pass Attempts',
 'def_tackles_for_loss',
 'rushing_tds',
 'Longest Completion',
 'Yards Per Punt Average',
 'Longest Punt (Yards)',
 'Passing Yards in First 10 Attempts',
 'Yards on First Pass Completion',
 'Quarters with 50+ Pass Yards',
 'Quarters with 25+ Pass Yards',
 'Quarters with 10+ Rush Yards',
 'Halves with 25+ Rush Yards',
 'Halves with 50+ Rush Yards',
 'Quarters with 1+ Reception',
 'Halves with 25+ Receiving Yards',
 'Punts',
 'Halves with 100+ Pass Yards',
 'Halves with 150+ Pass Yards',
 'qtrs_over_10_rec_yards',
 'Quarters with 20+ Rush Yards']

In [66]:
pp = get_pp_close()
pp = pp.loc[(pp.event_time >= pd.Timestamp.now('UTC')) & (pp.league_name == 'NFL') & (pp.scrape_time == max(pp['scrape_time']))]
pp

TypeError: '>=' not supported between instances of 'str' and 'Timestamp'

In [ ]:
schedule = con.execute('select * from schedule where season = 2025 and week = 2').df()

In [ ]:
pp.loc[pp.player == 'Michael Penix']

In [ ]:
today = pd.concat((
    pp.merge(schedule, left_on=['team'], right_on=['home_team']),
    pp.merge(schedule, left_on=['team'], right_on=['away_team'])
))
today['game_line'] = today.apply(lambda x: x['spread_line'] if x['home_team'] == x['team'] else -x['spread_line'],axis=1)
today['favored'] = today['game_line'].apply(lambda x: tiers(x))
today = today[['player','team','stat','alt_line','game_line']].merge(result, on=['stat','alt_line'])
today.loc[today.alt_line=='standard'].sort_values(by='over_ratio').reset_index(drop=True).iloc[0:25]

In [ ]:
today.loc[today.alt_line=='standard'].sort_values(by='over_ratio').reset_index(drop=True).tail(30)

In [ ]:
today.loc[(today.alt_line=='demon') & (today.stat != 'Sacks')].sort_values(by='over_ratio').tail(20)

In [ ]:
result.loc[(result['count'] > 10) &(result['alt_line']=='demon')].sort_values(by=['over_ratio'],ascending=False).drop(['over','push','under'],axis=1).head(30)

In [ ]:
result = pivot_lines(subset, ['alt_line', 'stat', 'pos']).sort_values(by='over_ratio', ascending=False)
result.loc[(result.alt_line == 'standard') & (result['count'] >= 15)].sort_values(by=['over_ratio'],ascending=False).tail(20)

In [ ]:
def get_corr(one, two,three=pd.DataFrame(), grouper=[], relation='same'):
    df1 = one.copy(deep=True)
    df2 = two.copy(deep=True)
    df3 = three.copy(deep=True)
    #get cols to merge on
    grouper = ['alt_line','stat'] + grouper
    home_cols = ['team', 'date']
    opp_cols = ['opp', 'date']
    if relation=='same':
        right_cols = home_cols
    else:
        right_cols = opp_cols

    #fixing weird column renaming
    if not df3.empty:
        a = 4
    else:
        a = 3
    count_cols = []
    o_u_cols = []
    for x in grouper:
        for n in range(1,a):
            count_cols += [x+f'_{n}']
            o_u_cols += [f'o_u_{n}']
    o_u_cols = sorted(list(set(o_u_cols)))

    raw_cols = list(set(grouper + home_cols + right_cols + ['o_u'] + ['player']))
    
    #merge dfs
    comb_corr = df1.loc[(df1.alt_line!='goblin') & (df1.o_u != 'push'), raw_cols].merge(
        df2.loc[(df2.alt_line!='goblin') & (df2.o_u != 'push'), raw_cols],
        left_on=home_cols,
        right_on=right_cols,
        suffixes=('_1', '_2'),
        )
    #filter out same player corr and mirrored lines
    comb_corr = comb_corr.loc[(comb_corr.player_1 != comb_corr.player_2) & 
                              ~((comb_corr.player_1 < comb_corr.player_2) & (comb_corr.stat_1 == comb_corr.stat_2))
                              ].reset_index(drop=True)
    if not df3.empty:
        comb_corr = comb_corr.merge(
            df3.loc[(df3.alt_line!='goblin'), raw_cols],
            left_on=home_cols,
            right_on=right_cols,
            #suffixes=('', '_2'),
            )
        comb_corr.columns = [x+'_3' if (x in raw_cols) else x for x in comb_corr.columns]
        comb_corr = comb_corr.loc[comb_corr.player_1 != comb_corr.player_3].reset_index(drop=True)
        comb_corr = comb_corr.loc[comb_corr.player_2 != comb_corr.player_3].reset_index(drop=True)
    corr = value_count_plus(
        comb_corr,
        count_cols,
        o_u_cols,
    )
    corr['relation'] = relation
    corr = corr.loc[~((corr.alt_line_1.isin(['demon', 'goblin'])) & (corr.o_u_1 == 'under'))]
    corr = corr.loc[~((corr.alt_line_2.isin(['demon', 'goblin'])) & (corr.o_u_2 == 'under'))]
    if not df3.empty:
        corr = corr.loc[~((corr.alt_line_3.isin(['demon', 'goblin'])) & (corr.o_u_3 == 'under'))]

    corr = corr.loc[corr['count'] > 0].sort_values(by='proportion', ascending=False)
    corr['equiv'] = corr['proportion'] ** (1/(a-1))
    corr = corr.rename({'count':'count_all'},axis=1)

    #filtering extraneous permutations
    numbered_cols = []
    for num in range(1,a):
        numbered_cols.append(num)
        corr[num] = ''
        for col in (['stat', 'alt_line', 'o_u'] + grouper):
            num_col = f'{col}_{num}'
            corr[num] += corr[num_col].astype(str)
    if a == 4:
        corr['concat'] = corr.apply(lambda x: str(sorted(list((x[1],x[2],x[3])))),axis=1)
    else:
        corr['concat'] = corr.apply(lambda x: str(sorted(list((x[1],x[2])))),axis=1)
    corr = corr.groupby(['concat']).head(1)
    corr = corr.sort_values(by='equiv',ascending=False).drop([1,2,3,'concat'],axis=1,errors='ignore')

    return corr.sort_values(by='proportion',ascending=False)

def weight_corr(corr, hit_df,suff):
    merge_cols = []
    for col in hit_df.columns:
        if 'hit_' in col:
            hit_col = col
        elif col != ('count') and col != ('proportion'):
            merge_cols.append(col)
    for n in range(1,4):
        n_merge_cols = [col + f'_{n}' for col in merge_cols]
        if f'alt_line_{n}' in corr.columns:
            corr = corr.merge(
                hit_df,
                left_on=n_merge_cols,
                right_on=merge_cols,
                how='left',
                suffixes=[f'_{n-1}',f'_{n}']
            )
            corr.drop(merge_cols, axis=1, inplace=True)
            corr.rename({f'hit_{suff}': f'hit_{suff}_{n}'},axis=1,inplace=True)
    corr['expected'] = 1
    for n in range(1,4):
        col = f'hit_{suff}_{n}'
        if col in corr.columns:
            corr['expected'] = corr['expected'] * corr[col]
    corr['diff'] = corr['proportion'] - corr['expected']
    return corr

hit_stat = value_count_plus(
	df.loc[df.o_u.isin(['under', 'over'])], ['alt_line', 'stat'], ['o_u']
)
#hit_stat['hit_stat'] = hit_stat.apply(lambda x: pad_results(x), axis=1)
hit_stat = hit_stat.rename({'proportion':'hit_stat'},axis=1)
hit_stat.head(3)



In [ ]:
corr = get_corr(df, df,df,  relation='same', grouper=['pos'])
corr = weight_corr(corr,hit_stat,'stat')
corr.loc[(corr.count_all > 55)].head(25)

In [ ]:
corr.loc[(corr.count_all > 55)].head(25)

In [ ]:
corr.loc[((
    (corr.pos_1=='QB') &
    (corr.pos_2=='K')) | 
    (
    (corr.pos_2=='QB') &
    (corr.pos_1=='K')
) )& (corr.count_all > 16)
].sort_values(by='diff',ascending=False).head(20)

In [ ]:
corr2 = get_corr(df, df, relation='same', grouper=['FantPos'])

In [ ]:
corr2.loc[(corr2.count_all > 15) & (corr2.stat_1=='FG') & (corr2.stat_2=='Pass Attempts')].head(25)

In [ ]:
corr2.loc[(corr2.stat_1=='Pass Yards') & (corr2.stat_2=='Fantasy Score') & (corr2.stat_3=='FG')]

In [ ]:
opp = get_corr(df, df, relation='opp',grouper=['FantPos'])
opp = opp.loc[(opp.count_all > 20) & ((opp.FantPos_1 =='QB') | (opp.FantPos_2 == 'QB'))]
opp.loc[(opp.count_all > 35)].head(20)

In [ ]:
wg = weight_corr(opp,hit_stat,'stat')
wg['diff'] = wg['proportion'] - wg['expected']
wg.loc[(wg.count_all > 80) & (wg.alt_line_1=='demon')].sort_values(by='diff',ascending=False).head(30)

In [ ]:
corr = get_corr(df, df, relation='same',grouper=['FantPos'])
corr.loc[(corr.count_all > 40) & (corr.stat_1 >= corr.stat_2)].head(30)
#corr.loc[(corr.count_all > 20) & (corr.stat_1 >= corr.stat_2) & (corr.alt_line_1 == 'demon')].head(30)

In [ ]:
corr2.loc[(corr2.stat_1.str.contains('TD')) & (corr2.stat_3.str.contains('TD')) & (corr2.FantPos_2 == 'K')]

In [ ]:
corr.loc[(corr.count_all > 20)& ((corr.alt_line_1 == 'demon') | (corr.alt_line_2 == 'demon'))].head(15)

In [ ]:
corr2.loc[(corr2.count_all > 25) & (
    (corr2.alt_line_1 == 'demon') |
    (corr2.alt_line_2 == 'demon') |
    (corr2.alt_line_3 == 'demon')) & (corr2.stat_1.str.contains("Fantasy")
)].head(30)

In [ ]:
corr2.loc[corr2.stat_1.str.contains('TD') & (corr2.count_all > 10)]

In [ ]:
corr2.to_csv('../reference_data/NFL/corr/corr3.csv')

In [ ]:
corr2.loc[(corr2.count_all > 35)].head(30)

In [ ]:
base = df[['player','team','line','alt_line','stat','opp','event_time','o_u']]
base

In [ ]:
pair =base.merge(base,on=['team','opp','event_time'])
pair = pair.loc[~(pair.player_x <= pair.player_y)]
pair.loc[(pair.player_x=='Dak Prescott') & (pair.player_y.str.contains('Lamb')) & (pair.stat_x=='Pass Yards') & (pair.stat_y=='Receiving Yards'),['o_u_x','o_u_y']].value_counts().reset_index()

In [ ]:
dk = pd.read_parquet('../Lines/dk/dk_2024_10_5.parquet')
dk.loc[dk['player'].str.contains('Chief',na=False)].head(5)

In [ ]:
play_corr = pair.groupby(['player_x','player_y','stat_x','stat_y','alt_line_x','alt_line_y'])[['o_u_x','o_u_y']].value_counts().reset_index()

In [ ]:
lovers = value_count_plus(
        play_corr.loc[play_corr.alt_line_x=='standard'],
        ['player_x','player_y','stat_x','stat_y','alt_line_x','alt_line_y'],
        ['o_u_x','o_u_y'],
    )

In [ ]:
lovers.loc[(lovers.player_x=='Dak Prescott') & (lovers.player_y.str.contains('Lamb')) & (lovers.stat_x=='Pass Yards') & (lovers.stat_y=='Receiving Yards')]

In [ ]:
pair.loc[(pair.player_x=='Dak Prescott') & (pair.player_y.str.contains('Lamb')) & (pair.stat_x=='Pass Yards') & (pair.stat_y=='Receiving Yards') & 
         (pair.alt_line_x=='standard') & (pair.alt_line_y=='standard')]

In [ ]:
common_cols = [ 'team', 'opp', 'home', 'roof', 'surface', 'weather', 'game_line', 'favored', 'total',
       'season',  'date']
qb_cols =     ['player', 'Pass Completions', 'Pass Attempts', 'Pass Yards',
       'Pass TDs', 'INT', 'Sacks', 'Yards Sacked', 'Longest Pass',
       'Passing Rate',  'snaps', 'pct', 'pos']
wr_cols =     ['player', 'Rec Targets', 'Receptions', 'Receiving Yards', 'Rush+Rec Yds', 
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fantasy Score',  'snaps', 'pct', 'pos']
rb_cols =     ['player', 'Rush Attempts', 'Rush Yards', 'Rush TDs', 'Rush+Rec Yds', 
       'Longest Rush', 'Rec Targets', 'Receptions', 'Receiving Yards',
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fumbles', 'Fumbles Lost', 'Fantasy Score', 'snaps', 'pct',
        'pos']

k_cols = ['player', 'XPM','Kicking Points', 'Fantasy Score', 'FG','snaps', 'pct', 'pos']

small = games.loc[games['date'] > '2014-09-01']

qb = small.loc[small.pos=='QB'][
    qb_cols + common_cols
]
wr = small.loc[small.pos=='WR'][
    wr_cols + common_cols
]
rb = small.loc[small.pos=='RB'][
    rb_cols + common_cols
]
te = small.loc[small.pos=='TE'][
    wr_cols + common_cols
]
k = small.loc[small.pos=='K'][
    k_cols + common_cols
]

In [ ]:
p1 = 'Jared Goff'
b = small.loc[small.player==p1]

p2 = 'Amon-Ra St. Brown'
t = small.loc[small.player==p2]

both = b.merge(t, on=common_cols)
print(len(both))
both.columns

In [ ]:
import matplotlib.pyplot as plt
stat = 'Pass Attempts'
stat1 = stat + '_x'
stat2 = stat + '_y'
stat2 = 'Rec Targets_y'

plt.axvline(x=both[stat1].median(), label=f'{p1} avg', color='r')
plt.axhline(y=both[stat2].median(), label=f'{p2} avg')
plt.scatter(both[stat1], both[stat2], c=both.index, cmap=plt.cm.coolwarm)
plt.xlabel(p1)
plt.ylabel(p2)
plt.title(stat)
plt.legend()

In [ ]:
both[[stat1,stat2]]